### Termos e Frequencia por ano e periodo

In [1]:
import nltk
import os
import os.path
import pandas as pd
from nltk.corpus import stopwords
stop = set(nltk.corpus.stopwords.words("portuguese"))

ini_path = 'E:/artigo3'
final_path = 'E:/artigo3/analises'
name_file = 'sentencas_classificadas_corrupcao.csv'
df = pd.read_csv(os.path.join(ini_path, name_file))
split_file = 'corrupcao_split.xlsx'
split = pd.read_excel(os.path.join(ini_path, split_file))
split = list(split[split.flag == 'interesse'].numero_cnj)
texto = list()

def remove_accents(text):
    accents = {
        "Á": "A","Ã": "A","À": "A","á": "a","ã": "a","à": "a",
        "É": "E","é": "e","Ê": "E","ê": "e","Í": "I","í": "i",
        "Ó": "O","ó": "o","Õ": "O","õ": "o","Ô": "O","ô": "o",
        "Ú": "U","ú": "u",";": "",",": "","/": "","\\": "",
        "{": "","}": "","(": "",")": "","-": "","_": "","Ç":"C",
        "ç":"c",
    }
    text = str(text)
    for k, v in accents.items():
        text = text.replace(k, v)
    return text

for ano in df.ano.unique():
    try:
        df1 = df[(df.ano == ano) & (df.numero_cnj.isin(split))]
        for _,row in df1.iterrows():
            try:
                decisao = remove_accents(row.decisao)
                texto = texto + [w.lower() for w in decisao.split() if w not in stop and w.isalpha() and len(w)>3]
            except:
                pass
        fdist1 = nltk.FreqDist(texto)
        termos1 = list()
        freq1 = list()
        for u,v in fdist1.items():
            termos1.append(u)
            freq1.append(v)
        final = pd.DataFrame()
        final['Termos'] = termos1
        final['Frequencia'] = freq1
        final.to_excel(os.path.join(final_path, f'freq_{ano}.xlsx'),index = False)
    except:
        pass

In [2]:
import nltk
import os
import os.path
import pandas as pd
from nltk.corpus import stopwords
stop = stopwords.words('spanish')+stopwords.words('english')+stopwords.words('portuguese')

ini_path = 'E:/artigo3'
final_path = 'E:/artigo3/analises'
name_file = 'sentencas_classificadas_corrupcao.csv'
df = pd.read_csv(os.path.join(ini_path, name_file))
split_file = 'corrupcao_split.xlsx'
split = pd.read_excel(os.path.join(ini_path, split_file))
split = list(split[split.flag == 'interesse'].numero_cnj)
texto = list()

def remove_accents(text):
    accents = {
        "Á": "A","Ã": "A","À": "A","á": "a","ã": "a","à": "a",
        "É": "E","é": "e","Ê": "E","ê": "e","Í": "I","í": "i",
        "Ó": "O","ó": "o","Õ": "O","õ": "o","Ô": "O","ô": "o",
        "Ú": "U","ú": "u",";": "",",": "","/": "","\\": "",
        "{": "","}": "","(": "",")": "","-": "","_": "","Ç":"C",
        "ç":"c",
    }
    text = str(text)
    for k, v in accents.items():
        text = text.replace(k, v)
    return text

df1 = df[(df.ano<=2014) & (df.numero_cnj.isin(split))]
for _,row in df1.iterrows():
    try:
        decisao = remove_accents(row.decisao)
        texto = texto + [w.lower() for w in decisao.split() if w not in stop and w.isalpha()]
    except:
        pass
fdist1 = nltk.FreqDist(texto)

df2 = df[(df.ano>2014) & (df.numero_cnj.isin(split))]
for _,row in df2.iterrows():
    try:
        decisao = remove_accents(row.decisao)
        texto = texto + [w.lower() for w in decisao.split() if w not in stop and w.isalpha()]
    except:
        pass
fdist2 = nltk.FreqDist(texto)

df3 = df[(df.ano>2018)& (df.numero_cnj.isin(split))]
for _,row in df3.iterrows():
    try:
        decisao = remove_accents(row.decisao)
        texto = texto + [w.lower() for w in decisao.split() if w not in stop and w.isalpha()]
    except:
        pass
fdist3 = nltk.FreqDist(texto)

termos1 = list()
freq1 = list()
termos2 = list()
freq2 = list()
termos3 = list()
freq3 = list()

for u,v in fdist1.items():
    termos1.append(u)
    freq1.append(v)

for u,v in fdist2.items():
    termos2.append(u)
    freq2.append(v)

for u,v in fdist3.items():
    termos3.append(u)
    freq3.append(v)

final = pd.DataFrame()
final['Termos_Antes'] = termos1
final['Freq_Antes'] = freq1
final.to_excel(os.path.join(final_path, 'freq_1.xlsx'),index = False)
final = pd.DataFrame()
final['Termos_Lava_Jato'] = termos2
final['Freq_Lava_Jato'] = freq2
final.to_excel(os.path.join(final_path, 'freq_2.xlsx'),index = False)
final = pd.DataFrame()
final['Termos_Vaza_Jato'] = termos3
final['Freq_Vaza_Jato'] = freq3
final.to_excel(os.path.join(final_path, 'freq_3.xlsx'),index = False)

# Classificação dos resultados dos processos

In [4]:
import re
import os
import os.path
import pandas as pd

ini_path = 'E:/artigo3/dados_trafico'
final_path = 'E:/artigo3'
file_list = os.listdir(ini_path)
df = pd.DataFrame()

for files in file_list:
    df = df.append(pd.read_csv(os.path.join(ini_path,files)), ignore_index=True)

df = df.dropna()

improcedente =list()
procedente = list()
parcialmente = list()
extinto = list()

for _,row in df.iterrows():
    resultado = [0, 0, 0, 0]
    resultadoProcesso = re.search(r"julg.{1,30}IMPROCEDE|Absolv[oa]", row.decisao, re.IGNORECASE)
    resultadoProcessop = re.search(r"julg.{1,30} PROCEDE|condeno|aplico.{1,20}pena", row.decisao, re.IGNORECASE)
    resultadoProcessopp = re.search(
        r"julg.{1,30}PARCIALMENTE\s*PROCEDENTE[S\s]?|homologo|homologando", row.decisao, re.IGNORECASE
    )
    resultadoExtincao = re.search(
        r"julg.{1,30}extin[guirtoa]?|extint[oa].{1,30}punibilidade|extint[oa]", row.decisao, re.IGNORECASE
    )
    if resultadoProcesso:
        resultado[0] = 1
    if resultadoProcessop:
        resultado[2] = 1
    if resultadoProcessopp or (resultado[2] and resultado[0]):
        resultado[1] = 1
        resultado[0] = 0
        resultado[2] = 0
    if (
        resultadoExtincao
        and resultado[0] == 0
        and resultado[1] == 0
        and resultado[2] == 0
    ):
        resultado[3] = 1

    if resultado[0] == 1:
        improcedente.append(1)
    else:
        improcedente.append(0)
    
    if resultado[1] == 1:
        parcialmente.append(1)
    else:
        parcialmente.append(0)
    
    if resultado[2] == 1:
        procedente.append(1)
    else:
        procedente.append(0)
    
    if resultado[3] == 1:
        extinto.append(1)
    else:
        extinto.append(0)
df['improcedente'] = improcedente
df['procedente'] = procedente
df['parcialmente'] = parcialmente
df['extinto'] = extinto
df['ano'] = df.data.str[-4:]
df.to_csv(os.path.join(final_path, f'sentencas_classificadas_trafico.csv'),index=False)

In [5]:
import re
import os
import os.path
import pandas as pd

ini_path = 'E:/artigo3/dados_corrupcao'
final_path = 'E:/artigo3'
file_list = os.listdir(ini_path)
df = pd.DataFrame()

for files in file_list:
    df = df.append(pd.read_csv(os.path.join(ini_path,files)), ignore_index=True)

df = df.dropna()

improcedente =list()
procedente = list()
parcialmente = list()
extinto = list()

for _,row in df.iterrows():
    resultado = [0, 0, 0, 0]
    resultadoProcesso = re.search(r"julg.{1,30}IMPROCEDE|Absolv[oa]", row.decisao, re.IGNORECASE)
    resultadoProcessop = re.search(r"julg.{1,30} PROCEDE|condeno|aplico.{1,20}pena", row.decisao, re.IGNORECASE)
    resultadoProcessopp = re.search(
        r"julg.{1,30}PARCIALMENTE\s*PROCEDENTE[S\s]?|homologo|homologando", row.decisao, re.IGNORECASE
    )
    resultadoExtincao = re.search(
        r"julg.{1,30}extin[guirtoa]?|extint[oa].{1,30}punibilidade|extint[oa]", row.decisao, re.IGNORECASE
    )
    if resultadoProcesso:
        resultado[0] = 1
    if resultadoProcessop:
        resultado[2] = 1
    if resultadoProcessopp or (resultado[2] and resultado[0]):
        resultado[1] = 1
        resultado[0] = 0
        resultado[2] = 0
    if (
        resultadoExtincao
        and resultado[0] == 0
        and resultado[1] == 0
        and resultado[2] == 0
    ):
        resultado[3] = 1

    if resultado[0] == 1:
        improcedente.append(1)
    else:
        improcedente.append(0)
    
    if resultado[1] == 1:
        parcialmente.append(1)
    else:
        parcialmente.append(0)
    
    if resultado[2] == 1:
        procedente.append(1)
    else:
        procedente.append(0)
    
    if resultado[3] == 1:
        extinto.append(1)
    else:
        extinto.append(0)
df['improcedente'] = improcedente
df['procedente'] = procedente
df['parcialmente'] = parcialmente
df['extinto'] = extinto
df['ano'] = df.data.str[-4:]
df.to_csv(os.path.join(final_path, f'sentencas_classificadas_corrupcao.csv'),index=False)

In [6]:
import re
import os
import os.path
import pandas as pd

ini_path = 'E:/artigo3/dados_crime'
final_path = 'E:/artigo3'
file_list = os.listdir(ini_path)
df = pd.DataFrame()

for files in file_list:
    df = df.append(pd.read_csv(os.path.join(ini_path,files)), ignore_index=True)

df = df.dropna()

improcedente =list()
procedente = list()
parcialmente = list()
extinto = list()

for _,row in df.iterrows():
    resultado = [0, 0, 0, 0]
    resultadoProcesso = re.search(r"julg.{1,30}IMPROCEDE|Absolv[oa]", row.decisao, re.IGNORECASE)
    resultadoProcessop = re.search(r"julg.{1,30} PROCEDE|condeno|aplico.{1,20}pena", row.decisao, re.IGNORECASE)
    resultadoProcessopp = re.search(
        r"julg.{1,30}PARCIALMENTE\s*PROCEDENTE[S\s]?|homologo|homologando", row.decisao, re.IGNORECASE
    )
    resultadoExtincao = re.search(
        r"julg.{1,30}extin[guirtoa]?|extint[oa].{1,30}punibilidade|extint[oa]", row.decisao, re.IGNORECASE
    )
    if resultadoProcesso:
        resultado[0] = 1
    if resultadoProcessop:
        resultado[2] = 1
    if resultadoProcessopp or (resultado[2] and resultado[0]):
        resultado[1] = 1
        resultado[0] = 0
        resultado[2] = 0
    if (
        resultadoExtincao
        and resultado[0] == 0
        and resultado[1] == 0
        and resultado[2] == 0
    ):
        resultado[3] = 1

    if resultado[0] == 1:
        improcedente.append(1)
    else:
        improcedente.append(0)
    
    if resultado[1] == 1:
        parcialmente.append(1)
    else:
        parcialmente.append(0)
    
    if resultado[2] == 1:
        procedente.append(1)
    else:
        procedente.append(0)
    
    if resultado[3] == 1:
        extinto.append(1)
    else:
        extinto.append(0)
df['improcedente'] = improcedente
df['procedente'] = procedente
df['parcialmente'] = parcialmente
df['extinto'] = extinto
df['ano'] = df.data.str[-4:]
df.to_csv(os.path.join(final_path, f'sentencas_classificadas_crime.csv'),index=False)

# Pre-Processamento das Sentenças

In [4]:
import re
import os
import os.path
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import nltk

ini_path = 'E:/artigo3'
files = 'sentencas_classificadas_corrupcao.csv'
df = pd.read_csv(os.path.join(ini_path,files))

def remove_accents(text):
    accents = {
        "Á": "A","Ã": "A","À": "A","á": "a","ã": "a","à": "a",
        "É": "E","é": "e","Ê": "E","ê": "e","Í": "I","í": "i",
        "Ó": "O","ó": "o","Õ": "O","õ": "o","Ô": "O","ô": "o",
        "Ú": "U","ú": "u",";": "",",": "","/": "","\\": "",
        "{": "","}": "","(": "",")": "","-": "","_": "","Ç":"C",
        "ç":"c",
    }
    text = str(text)
    for k, v in accents.items():
        text = text.replace(k, v)
    return text

def normalize_texts(texts, to_stem=False):
    if to_stem:
        stemmer = nltk.stem.RSLPStemmer()
    normal_texts = []
    tk = RegexpTokenizer(r"\w+")
    stopwords = set(nltk.corpus.stopwords.words("portuguese"))
    for t in texts:
        raw_text = remove_accents(t.lower())  # steps 1 and 2
        tokens = tk.tokenize(raw_text)  # step 3
        processed_text = ""
        for tkn in tokens:
            if tkn.isalpha() and tkn not in stopwords and len(tkn) > 3:  # step 4
                if to_stem:
                    tkn = stemmer.stem(tkn)  # step 5
                processed_text += tkn + " "
        normal_texts.append(processed_text[:-1])
    return normal_texts

decisao_clean = list()
for _,row in df.iterrows():
    decisao_clean.append(normalize_texts([row.decisao], to_stem=True))
df['decisao_clean'] = decisao_clean

decisao_clean = list()
for _,row in df.iterrows():
    decisao_clean.append(row.decisao_clean[0])
df['decisao_clean'] = decisao_clean
df.to_csv(os.path.join(ini_path, f'sentencas_corrupcao_processadas.csv'),index=False)

In [5]:
import re
import os
import os.path
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import nltk

ini_path = 'E:/artigo3'
files = 'sentencas_classificadas_trafico.csv'
df = pd.read_csv(os.path.join(ini_path,files))

def remove_accents(text):
    accents = {
        "Á": "A","Ã": "A","À": "A","á": "a","ã": "a","à": "a",
        "É": "E","é": "e","Ê": "E","ê": "e","Í": "I","í": "i",
        "Ó": "O","ó": "o","Õ": "O","õ": "o","Ô": "O","ô": "o",
        "Ú": "U","ú": "u",";": "",",": "","/": "","\\": "",
        "{": "","}": "","(": "",")": "","-": "","_": "","Ç":"C",
        "ç":"c",
    }
    text = str(text)
    for k, v in accents.items():
        text = text.replace(k, v)
    return text

def normalize_texts(texts, to_stem=False):
    if to_stem:
        stemmer = nltk.stem.RSLPStemmer()
    normal_texts = []
    tk = RegexpTokenizer(r"\w+")
    stopwords = set(nltk.corpus.stopwords.words("portuguese"))
    for t in texts:
        raw_text = remove_accents(t.lower())  # steps 1 and 2
        tokens = tk.tokenize(raw_text)  # step 3
        processed_text = ""
        for tkn in tokens:
            if tkn.isalpha() and tkn not in stopwords and len(tkn) > 3:  # step 4
                if to_stem:
                    tkn = stemmer.stem(tkn)  # step 5
                processed_text += tkn + " "
        normal_texts.append(processed_text[:-1])
    return normal_texts

decisao_clean = list()
for _,row in df.iterrows():
    decisao_clean.append(normalize_texts([row.decisao], to_stem=True))
df['decisao_clean'] = decisao_clean

decisao_clean = list()
for _,row in df.iterrows():
    decisao_clean.append(row.decisao_clean[0])
df['decisao_clean'] = decisao_clean
df.to_csv(os.path.join(ini_path, f'sentencas_trafico_processadas.csv'),index=False)

In [6]:
import re
import os
import os.path
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import nltk

ini_path = 'E:/artigo3'
files = 'sentencas_classificadas_crime.csv'
df = pd.read_csv(os.path.join(ini_path,files))

def remove_accents(text):
    accents = {
        "Á": "A","Ã": "A","À": "A","á": "a","ã": "a","à": "a",
        "É": "E","é": "e","Ê": "E","ê": "e","Í": "I","í": "i",
        "Ó": "O","ó": "o","Õ": "O","õ": "o","Ô": "O","ô": "o",
        "Ú": "U","ú": "u",";": "",",": "","/": "","\\": "",
        "{": "","}": "","(": "",")": "","-": "","_": "","Ç":"C",
        "ç":"c",
    }
    text = str(text)
    for k, v in accents.items():
        text = text.replace(k, v)
    return text

def normalize_texts(texts, to_stem=False):
    if to_stem:
        stemmer = nltk.stem.RSLPStemmer()
    normal_texts = []
    tk = RegexpTokenizer(r"\w+")
    stopwords = set(nltk.corpus.stopwords.words("portuguese"))
    for t in texts:
        raw_text = remove_accents(t.lower())  # steps 1 and 2
        tokens = tk.tokenize(raw_text)  # step 3
        processed_text = ""
        for tkn in tokens:
            if tkn.isalpha() and tkn not in stopwords and len(tkn) > 3:  # step 4
                if to_stem:
                    tkn = stemmer.stem(tkn)  # step 5
                processed_text += tkn + " "
        normal_texts.append(processed_text[:-1])
    return normal_texts

decisao_clean = list()
for _,row in df.iterrows():
    decisao_clean.append(normalize_texts([row.decisao], to_stem=True))
df['decisao_clean'] = decisao_clean

decisao_clean = list()
for _,row in df.iterrows():
    decisao_clean.append(row.decisao_clean[0])
df['decisao_clean'] = decisao_clean
df.to_csv(os.path.join(ini_path, f'sentencas_crime_processadas.csv'),index=False)

In [ ]:
import re
import os
import os.path
import pandas as pd
import numpy as np
import string
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn_extra.cluster import KMedoids
from sklearn.neighbors import DistanceMetric

ini_path = 'E:/artigo3'
files = 'sentencas_corrupcao_processadas.csv'
corrupcao = pd.read_csv(os.path.join(ini_path,files))
corrupcao = corrupcao.dropna()
centroids = list()

def closest_n_index(X, n):
    kmedoids = KMedoids(n_clusters=n, random_state=0).fit(X)
    return kmedoids.cluster_centers_, kmedoids.labels_

n_features = 10000
vectorizer_hash = HashingVectorizer(n_features=n_features, ngram_range=(1, 3))
X_hash = vectorizer_hash.transform(corrupcao.decisao_clean)
corrupcao['vector'] = list(X_hash.toarray())
corrupcao['dia'] = corrupcao.data.str[:2]
corrupcao['mes'] = corrupcao.data.str[3:5]
corrupcao['ano'] = corrupcao.data.str[6:11]
corrupcao.sort_values(by=['comarca','ano','mes','dia'], inplace=True)

for comarca in corrupcao.comarca.unique():
    for year in corrupcao.ano.unique():
        for month in corrupcao.mes.unique():
            if comarca == 'SÃO PAULO':
                for foro in corrupcao[corrupcao.comarca == comarca].foro.unique():
                    try:
                        X_hash = vectorizer_hash.transform(corrupcao.decisao_clean[(corrupcao.comarca == comarca) & (corrupcao.foro == foro) & (corrupcao.ano == year) & (corrupcao.mes == month)])
                        n_assunto = len(corrupcao.assunto[(corrupcao.comarca == comarca) & (corrupcao.foro == foro) & (corrupcao.ano == year) & (corrupcao.mes == month)].unique())
                        indices_centers, labels = closest_n_index(X_hash, n_assunto)
                        for vector in indices_centers.toarray():
                            rows = dict()
                            rows['comarca'] = comarca + ' - '+ foro
                            rows['ano'] = year
                            rows['mes'] = month
                            rows['vectors'] = vector
                            centroids.append(rows)
                    except:
                        rows = dict()
                        rows['comarca'] = comarca + ' - '+ foro
                        rows['ano'] = year
                        rows['mes'] = month
                        rows['vectors'] = np.nan
                        centroids.append(rows)
            
            if comarca != 'SÃO PAULO':
                try:
                    X_hash = vectorizer_hash.transform(corrupcao.decisao_clean[(corrupcao.comarca == comarca) & (corrupcao.ano == year) & (corrupcao.mes == month)])
                    n_assunto = len(corrupcao.assunto[(corrupcao.comarca == comarca) & (corrupcao.ano == year) & (corrupcao.mes == month)].unique())
                    indices_centers, labels = closest_n_index(X_hash, n_assunto)
                    for vector in indices_centers.toarray():
                        rows = dict()
                        rows['comarca'] = comarca
                        rows['ano'] = year
                        rows['mes'] = month
                        rows['vectors'] = vector
                        centroids.append(rows)
                except:
                    rows = dict()
                    rows['comarca'] = comarca
                    rows['ano'] = year
                    rows['mes'] = month
                    rows['vectors'] = np.nan
                    centroids.append(rows)

corrupcao = pd.DataFrame(centroids)
corrupcao = corrupcao.dropna()

n_features = 10000
vectorizer_hash = HashingVectorizer(n_features=n_features, ngram_range=(1, 3))

files = 'sentencas_trafico_processadas.csv'
trafico = pd.read_csv(os.path.join(ini_path,files))
trafico = trafico.dropna()

X_hash = vectorizer_hash.transform(trafico.decisao_clean)
trafico['vector'] = list(X_hash.toarray())
trafico['dia'] = trafico.data.str[:2]
trafico['mes'] = trafico.data.str[3:5]
trafico['ano'] = trafico.data.str[6:11]
trafico.sort_values(by=['comarca','ano','mes','dia'], inplace=True)
centroids = list()

for comarca in trafico.comarca.unique():
    for year in trafico.ano.unique():
        for month in trafico.mes.unique():
            if comarca == 'SÃO PAULO':
                for foro in trafico[trafico.comarca == comarca].foro.unique():
                    try:
                        X_hash = vectorizer_hash.transform(trafico.decisao_clean[(trafico.comarca == comarca) & (trafico.foro == foro) & (trafico.ano == year) & (trafico.mes == month)])
                        n_assunto = len(trafico.assunto[(trafico.comarca == comarca) & (trafico.foro == foro) & (trafico.ano == year) & (trafico.mes == month)].unique())
                        indices_centers, labels = closest_n_index(X_hash, n_assunto)
                        for vector in indices_centers.toarray():
                            rows = dict()
                            rows['comarca'] = comarca + ' - '+ foro
                            rows['ano'] = year
                            rows['mes'] = month
                            rows['vectors'] = vector
                            centroids.append(rows)
                    except:
                        rows = dict()
                        rows['comarca'] = comarca + ' - '+ foro
                        rows['ano'] = year
                        rows['mes'] = month
                        rows['vectors'] = np.nan
                        centroids.append(rows)
            
            if comarca != 'SÃO PAULO':
                try:
                    X_hash = vectorizer_hash.transform(trafico.decisao_clean[(trafico.comarca == comarca) & (trafico.ano == year) & (trafico.mes == month)])
                    n_assunto = len(trafico.assunto[(trafico.comarca == comarca) & (trafico.ano == year) & (trafico.mes == month)].unique())
                    indices_centers, labels = closest_n_index(X_hash, n_assunto)
                    for vector in indices_centers.toarray():
                        rows = dict()
                        rows['comarca'] = comarca
                        rows['ano'] = year
                        rows['mes'] = month
                        rows['vectors'] = vector
                        centroids.append(rows)
                except:
                    rows = dict()
                    rows['comarca'] = comarca
                    rows['ano'] = year
                    rows['mes'] = month
                    rows['vectors'] = np.nan
                    centroids.append(rows)

trafico = pd.DataFrame(centroids)
trafico = trafico.dropna()
centroids = list()

dist = DistanceMetric.get_metric('euclidean')
distancia = list()
distancia_min = list()

for _,row in corrupcao.iterrows():
    X = row.vectors
    num_anterior = list()
    if row.comarca == 'SÃO PAULO':
        for linha in list(trafico[(trafico.comarca == row.comarca) & (trafico.foro == row.foro) & (trafico.ano == row.ano) & (trafico.mes == row.mes)].vectors):
            compare = list()
            compare.append(X)
            compare.append(linha)
            num_anterior.append(dist.pairwise(compare)[0][1])
        distancia.append(num_anterior)
    
    if row.comarca != 'SÃO PAULO':
        for linha in list(trafico[(trafico.comarca == row.comarca) & (trafico.ano == row.ano) & (trafico.mes == row.mes)].vectors):
            compare = list()
            compare.append(X)
            compare.append(linha)
            num_anterior.append(dist.pairwise(compare)[0][1])
        distancia.append(num_anterior)
        
corrupcao['distancia'] = distancia
corrupcao = corrupcao.drop('vectors',axis='columns', inplace=True)
corrupcao.to_excel(os.path.join(ini_path, f'corrupcao_trafico_distancia.xlsx'), index=False)

In [ ]:
import re
import os
import os.path
import pandas as pd
import numpy as np
import string
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn_extra.cluster import KMedoids
from sklearn.neighbors import DistanceMetric

ini_path = 'E:/artigo3'
files = 'sentencas_corrupcao_processadas.csv'
corrupcao = pd.read_csv(os.path.join(ini_path,files))
corrupcao = corrupcao.dropna()
centroids = list()

def closest_n_index(X, n):
    kmedoids = KMedoids(n_clusters=n, random_state=0).fit(X)
    return kmedoids.cluster_centers_, kmedoids.labels_

n_features = 10000
vectorizer_hash = HashingVectorizer(n_features=n_features, ngram_range=(1, 3))
X_hash = vectorizer_hash.transform(corrupcao.decisao_clean)
corrupcao['vector'] = list(X_hash.toarray())
corrupcao['dia'] = corrupcao.data.str[:2]
corrupcao['mes'] = corrupcao.data.str[3:5]
corrupcao['ano'] = corrupcao.data.str[6:11]
corrupcao.sort_values(by=['comarca','ano','mes','dia'], inplace=True)

for comarca in corrupcao.comarca.unique():
    for year in corrupcao.ano.unique():
        for month in corrupcao.mes.unique():
            if comarca == 'SÃO PAULO':
                for foro in corrupcao[corrupcao.comarca == comarca].foro.unique():
                    try:
                        X_hash = vectorizer_hash.transform(corrupcao.decisao_clean[(corrupcao.comarca == comarca) & (corrupcao.foro == foro) & (corrupcao.ano == year) & (corrupcao.mes == month)])
                        n_assunto = len(corrupcao.assunto[(corrupcao.comarca == comarca) & (corrupcao.foro == foro) & (corrupcao.ano == year) & (corrupcao.mes == month)].unique())
                        indices_centers, labels = closest_n_index(X_hash, n_assunto)
                        for vector in indices_centers.toarray():
                            rows = dict()
                            rows['comarca'] = comarca + ' - '+ foro
                            rows['ano'] = year
                            rows['mes'] = month
                            rows['vectors'] = vector
                            centroids.append(rows)
                    except:
                        rows = dict()
                        rows['comarca'] = comarca + ' - '+ foro
                        rows['ano'] = year
                        rows['mes'] = month
                        rows['vectors'] = np.nan
                        centroids.append(rows)
            
            if comarca != 'SÃO PAULO':
                try:
                    X_hash = vectorizer_hash.transform(corrupcao.decisao_clean[(corrupcao.comarca == comarca) & (corrupcao.ano == year) & (corrupcao.mes == month)])
                    n_assunto = len(corrupcao.assunto[(corrupcao.comarca == comarca) & (corrupcao.ano == year) & (corrupcao.mes == month)].unique())
                    indices_centers, labels = closest_n_index(X_hash, n_assunto)
                    for vector in indices_centers.toarray():
                        rows = dict()
                        rows['comarca'] = comarca
                        rows['ano'] = year
                        rows['mes'] = month
                        rows['vectors'] = vector
                        centroids.append(rows)
                except:
                    rows = dict()
                    rows['comarca'] = comarca
                    rows['ano'] = year
                    rows['mes'] = month
                    rows['vectors'] = np.nan
                    centroids.append(rows)

corrupcao = pd.DataFrame(centroids)
corrupcao = corrupcao.dropna()

n_features = 10000
vectorizer_hash = HashingVectorizer(n_features=n_features, ngram_range=(1, 3))

files = 'sentencas_crime_processadas.csv'
trafico = pd.read_csv(os.path.join(ini_path,files))
trafico = trafico.dropna()

X_hash = vectorizer_hash.transform(trafico.decisao_clean)
trafico['vector'] = list(X_hash.toarray())
trafico['dia'] = trafico.data.str[:2]
trafico['mes'] = trafico.data.str[3:5]
trafico['ano'] = trafico.data.str[6:11]
trafico.sort_values(by=['comarca','ano','mes','dia'], inplace=True)
centroids = list()

for comarca in trafico.comarca.unique():
    for year in trafico.ano.unique():
        for month in trafico.mes.unique():
            if comarca == 'SÃO PAULO':
                for foro in trafico[trafico.comarca == comarca].foro.unique():
                    try:
                        X_hash = vectorizer_hash.transform(trafico.decisao_clean[(trafico.comarca == comarca) & (trafico.foro == foro) & (trafico.ano == year) & (trafico.mes == month)])
                        n_assunto = len(trafico.assunto[(trafico.comarca == comarca) & (trafico.foro == foro) & (trafico.ano == year) & (trafico.mes == month)].unique())
                        indices_centers, labels = closest_n_index(X_hash, n_assunto)
                        for vector in indices_centers.toarray():
                            rows = dict()
                            rows['comarca'] = comarca + ' - '+ foro
                            rows['ano'] = year
                            rows['mes'] = month
                            rows['vectors'] = vector
                            centroids.append(rows)
                    except:
                        rows = dict()
                        rows['comarca'] = comarca + ' - '+ foro
                        rows['ano'] = year
                        rows['mes'] = month
                        rows['vectors'] = np.nan
                        centroids.append(rows)
            
            if comarca != 'SÃO PAULO':
                try:
                    X_hash = vectorizer_hash.transform(trafico.decisao_clean[(trafico.comarca == comarca) & (trafico.ano == year) & (trafico.mes == month)])
                    n_assunto = len(trafico.assunto[(trafico.comarca == comarca) & (trafico.ano == year) & (trafico.mes == month)].unique())
                    indices_centers, labels = closest_n_index(X_hash, n_assunto)
                    for vector in indices_centers.toarray():
                        rows = dict()
                        rows['comarca'] = comarca
                        rows['ano'] = year
                        rows['mes'] = month
                        rows['vectors'] = vector
                        centroids.append(rows)
                except:
                    rows = dict()
                    rows['comarca'] = comarca
                    rows['ano'] = year
                    rows['mes'] = month
                    rows['vectors'] = np.nan
                    centroids.append(rows)

trafico = pd.DataFrame(centroids)
trafico = trafico.dropna()
centroids = list()

dist = DistanceMetric.get_metric('euclidean')
distancia = list()

for _,row in corrupcao.iterrows():
    X = row.vectors
    num_anterior = list()
    if row.comarca == 'SÃO PAULO':
        for linha in list(trafico[(trafico.comarca == row.comarca) & (trafico.foro == row.foro) & (trafico.ano == row.ano) & (trafico.mes == row.mes)].vectors):
            compare = list()
            compare.append(X)
            compare.append(linha)
            num_anterior.append(dist.pairwise(compare)[0][1])
        distancia.append(num_anterior)
    
    if row.comarca != 'SÃO PAULO':
        for linha in list(trafico[(trafico.comarca == row.comarca) & (trafico.ano == row.ano) & (trafico.mes == row.mes)].vectors):
            compare = list()
            compare.append(X)
            compare.append(linha)
            num_anterior.append(dist.pairwise(compare)[0][1])
        distancia.append(num_anterior)
        
corrupcao['distancia'] = distancia
corrupcao = corrupcao.drop('vectors',axis='columns', inplace=True)
corrupcao.to_excel(os.path.join(ini_path, f'corrupcao_trafico_distancia.xlsx'), index=False)

In [1]:
import os
import os.path
from io import StringIO
import pandas as pd
import re
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

def convert_pdf_to_string(file_path):
    output_string = StringIO()
    with open(file_path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)
    return (output_string.getvalue())

ini_path = 'F:/trf4/pdfs'
final_path = 'F:/trf4/csvs'

re_num_cnj = r"\d{4,8}\s*\-*\.*\s*\d{2}\s*\.*\s*\d{4}\s*\.*\s*\d{1}\s*\.*\s*\d{2}\s*\.*\s*\d{4}"
re_num_trf_trt = r"\d{4}\.\d{2}\.\d{2}\.\d{6}\-\d|\d{7}\s*?-\d{2}\s*?\.\d{4}\s*?\.\d{1}\s*?\.\d{2}\s*?\.\d{4}|\d{7}\s*?-\d{2}\s*?\.\d{4}\s*?\.\d{3}\s*?\.\d{4}|\d{15}|\d{3,5}\-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4}"
re_final_rj = [r"(\n\n\s*?)(Proc\.)",]
re_final_rs = [
            r"(\n\n\s*?)(EDITAL DE)",
            r"(\n\n\s*?)(\d{7})",
            r"(\n\n\s*?)(\d+.*?\(?\s*?CNJ)",
        ]
re_final_sc = [
    r"(\n\n\s*?)(ADV\s*?\:)",
    r"(\n\n\s*?)(Processo)",
    r"(\n\n\s*?)(\d*\s*\.*Recurso)",
    r"(\n\n\s*?)(\d*\s*\.*Ag\s*ra\s*vo)",
    r"(\n\n\s*?)(\d*\s*\.*Embargo)",
    r"(\n\n\s*?)(\d*\s*\.*Apelação)",
    r"(\n\n\s*?)(N\.)",
    r"(\n\n\s*?)(\d+\s*?\-\s*?N\.)",
]
re_final_trf1 = [
    r"(\n\n\s*?)(Processo Orig)",
    r"(\n\n\s*?)(Tribunal\s*?Regional\s*?Federal\s*?da\s*?1ª\s*?Região)",
    r"(\n\n\s*?)(PODER JUDICI.RIO)",
    r"(\n\n\s*?)(Numera..o .nica)",
    r"(\n\n\s*?)(\d{4,8})",
    r"(\n\n\s*?)(Ap\s)",
    r"(\n\n\s*?)(APELAÇÃO\s*?C.VEL)",
    r"(\n\n\s*?)(AGRAVO\s*?DE\s*?INSTRUMENTO)",
    r"(\n\n\s*?)(N\.)",
    r"(\n\n\s*?)(REEXAME\s*?NECESSÁRIO\s*?N\.)",
    r"(\n\n\s*?)(APELAÇÃO\/REEXAME\s*?NECESSÁRIO)",
    r"(\n\n\s*?)(Relator\s*?AGRAVO\s*?DE\s*?INSTRUMENTO)",
]
re_final_trf4 = [
    r"(\n\n\s*?)(AGRAVO)",
    r"(\n\n\s*?)(MANDADO)",
    r"(\n\n\s*?)(EXECUÇÃO)",
    r"(\n\n\s*?)(PROCEDIMENTO)",
    r"(\n\n\s*?)(AÇÃO)",
    r"(\n\n\s*?)(REMESSA)",
    r"(\n\n\s*?)(APELAÇÃO)",
    r"(\n\n\s*?)(EDITAL)",
    r"(\n\n\s*?)(\d{7})",
]
re_final_trf5 = [
    r"(\n\n\s*?)(AC \-)",
    r"(\n\n\s*?)(AGTR)",
    r"(\n\n\s*?)(REOAC)",
    r"(\n\n\s*?)(APELREEX)",
    r"(\n\n\s*?)()",
    r"(\n\n\s*?)(AGIVP)",
    r"(\n\n\s*?)(PROTOCOLO)",
    r"(\n\n\s*?)(\d{4}\s*?\.\s*Processo)",
]

In [2]:
texto = convert_pdf_to_string(os.path.join(ini_path, 'TFR4_10.pdf'))
texto

'DIÁRIO ELETRÔNICO DA JUSTIÇA FEDERAL DA 4ª REGIÃO\nAno I – nº 5 – Porto Alegre, quarta-feira, 08 de novembro de 2006\n\nTRIBUNAL REGIONAL FEDERAL DA 4ª REGIÃO\n\nPUBLICAÇÕES JUDICIAIS\n\nSECRETARIA DE RECURSOS\n\nTRIBUNAL REGIONAL FEDERAL DA 4ª REGIÃO\n\nExpediente Recursos Nro 8567/2006\n\nSecretaria de Recursos\n\nNos processos abaixo relacionados o Excelentíssimo Senhor Desembargador Federal Amaury Chaves de Athayde, Vice-Presidente\ndo Tribunal Regional Federal da 4ª Região, proferiu os seguintes despachos (observação: o despacho se encontra abaixo de cada\nprocesso): \n00001 RECURSO ESPECIAL EM AC Nº 2001.71.02.005138-1/RS\nRECTE \nADVOGADO\nRECDOS \nADVOGADOS\n\n: UNIÃO \n: Luis Henrique Martins dos Anjos\n: AMARILDO LUIZ TREVISAN e outros\n:\n\nJose Luis Wagner e outros\n\nDECISÃO\nAnte o exposto, admito o recurso especial.\n00002 RECURSO EXTRAORDINÁRIO EM AC Nº 2001.71.02.005138-1/RS\nRECTE \nADVOGADO\nRECDOS \nADVOGADOS\n\n: UNIÃO \n: Luis Henrique Martins dos Anjos\n: AMARIL

In [8]:
for separador in re_final_trf4:
    texto = re.sub(separador, r"@@@@", texto)
publicacoes = [re.sub(r"\s+", " ", i) for i in re.split(r"@@@@", texto)[1:-1] if i and len(i) > 100]

In [10]:
publicacoes[0]

'APELAÇÃO EM MANDADO DE SEGURANÇA. TRIBUTÁRIO. RECURSO ADMINISTRATIVO. DEPÓSITO DE 30% DO VALOR DO DÉBITO. ARROLAMENTO DE BENS. 1. A exigência do depósito prévio de 30% do valor da exigência fiscal para seguimento ao recurso administrativo não afronta os princípios constitucionais da ampla defesa e do contraditório, nem implica violação ao direito de petição, porquanto não podem ser entendidos como princípios impeditivos à instituição de requisitos ou condições para o processamento de recursos administrativos. 2. A exigência de depósito recursal prévio não ofende o artigo 151, inciso III, do CTN, porquanto, se preenchidos os requisitos legais de admissibilidade do recurso administrativo, entre os quais o depósito prévio, a conseqüência necessária daí adveniente é a suspensão da exigibilidade do crédito tributário. 3. O Decreto nº 3.048/99 (RPS), no seu artigo 304, prevê a aplicação subsidiária do Decreto nº 70.235/72, assim, em não havendo incompatibilidade, incidem sobre os procedimen

In [5]:
re.sub(r"(\n\n\s*?)(AÇÃO)", r"\1@@@@\2", texto)

'DIÁRIO ELETRÔNICO DA JUSTIÇA FEDERAL DA 4ª REGIÃO\nAno I – nº 5 – Porto Alegre, quarta-feira, 08 de novembro de 2006\n\nTRIBUNAL REGIONAL FEDERAL DA 4ª REGIÃO\n\nPUBLICAÇÕES JUDICIAIS\n\nSECRETARIA DE RECURSOS\n\nTRIBUNAL REGIONAL FEDERAL DA 4ª REGIÃO\n\nExpediente Recursos Nro 8567/2006\n\nSecretaria de Recursos\n\nNos processos abaixo relacionados o Excelentíssimo Senhor Desembargador Federal Amaury Chaves de Athayde, Vice-Presidente\ndo Tribunal Regional Federal da 4ª Região, proferiu os seguintes despachos (observação: o despacho se encontra abaixo de cada\nprocesso): \n00001 RECURSO ESPECIAL EM AC Nº 2001.71.02.005138-1/RS\nRECTE \nADVOGADO\nRECDOS \nADVOGADOS\n\n: UNIÃO \n: Luis Henrique Martins dos Anjos\n: AMARILDO LUIZ TREVISAN e outros\n:\n\nJose Luis Wagner e outros\n\nDECISÃO\nAnte o exposto, admito o recurso especial.\n00002 RECURSO EXTRAORDINÁRIO EM AC Nº 2001.71.02.005138-1/RS\nRECTE \nADVOGADO\nRECDOS \nADVOGADOS\n\n: UNIÃO \n: Luis Henrique Martins dos Anjos\n: AMARIL

In [ ]:
for files in os.listdir(ini_path):
    if files.endswith('.pdf'):
        print(files)
        try:
            texto = convert_pdf_to_string(os.path.join(ini_path, files))
            data_publicacao = re.search(r'(\d{1})\sde\s(\w{4,8})\sde\s(\d{4})|(\d{2})\sde\s(\w{4,8})\sde\s(\d{4})',texto).groups()
            if data_publicacao[0] == None:
                dia = data_publicacao[3]
            else:
                dia = data_publicacao[0]

            if data_publicacao[1] == None:
                mes = data_publicacao[4]
            else:
                mes = data_publicacao[1]

            if data_publicacao[2] == None:
                ano = data_publicacao[5]
            else:
                ano = data_publicacao[2]
            
            instancia = 'TRF4'
            for separador in re_final_trf4:
                texto_corte = re.sub(separador, r"\1@@@@\2", texto)
            publicacoes = [re.sub(r"\s+", " ", i) for i in re.split(r"@@@@", texto_corte)[1:-1] if i and len(i) > 100]
            rows = list()

            for pub in publicacoes:
                linha = dict()
                linha['dia'] = dia
                linha['mes'] = mes
                linha['ano'] = ano                 
                linha['instancia'] = instancia
                if re.search(re_num_trf_trt,pub):
                    linha['numero_cnj'] = re.search(re_num_trf_trt,pub).group()
                else:
                    linha['numero_cnj'] = None
                linha['publicacao'] = pub
                rows.append(linha)
            final = pd.DataFrame(rows)
            final.to_csv(os.path.join(final_path, f'{files[:-4]}.csv'),index=False)
        except:
            pass